In [15]:
__author__ = "me"
__date__ = "2016_02_01"

%pylab inline
import sqlite3
import datetime
import dateutil.parser
import os
import ast


Populating the interactive namespace from numpy and matplotlib


In [16]:

def getFiles(folder, format, full=True, filtr=None):
    for f in os.listdir(folder):
        if f.endswith(format):
            if filtr==None or filtr not in f:
                if full:
                    yield os.path.join(folder, f)
                else:
                    yield f

In [17]:
def getSQLiteTweets(path):
    conn = sqlite3.connect(path)
    df = conn.cursor().execute('SELECT id, tweet, geo, timestamp, user_id FROM tweets').fetchall() ## all but raw data
    conn.close()
    return df

In [4]:
def parseRow(row, ID='?'):
    '''parse row to the defined schema'''
    try:
        geo =  ast.literal_eval(row[2])['coordinates']              # some don't have geocoordinates
        return (int(row[0]),                                        # tweet id
                int(dateutil.parser.parse(row[3]).strftime("%s")),  # timestamp
                geo[0],                                             # lon (geo)
                geo[1],                                             # lat (geo)
                row[1],                                             # text body
                int(row[4]),                                        # user id
                None,                                               # retweets count
                None,                                               # favorites count
                None,                                               # source app count
                ID                                                  # source_scraper
                )
    except:
        pass


In [9]:
def parseDB(path):
    db = getSQLiteTweets(path)
    
    ID=unicode(path.split('/')[-1])
    
    result = []
    for x in db:
        i = parseRow(x, ID=ID)
        if i!=None:
            result.append(i)
    return result

In [7]:
tweetE = '''CREATE TABLE IF NOT EXISTS tweets (id INTEGER PRIMARY KEY, 
                                               timestamp INTEGER, 
                                               lon REAL, lat REAL, 
                                               tweet TEXT, 
                                               user_id INTEGER, 
                                               rtwts INTEGER,
                                               fvrts INTEGER,
                                               application TEXT,
                                               source TEXT )'''

In [8]:
def merge_scraped(path):
    dbPath = path + 'MERGED_SCRAPER.db'

    conn = sqlite3.connect(dbPath)
    c = conn.cursor()
    c.execute(tweetE)
    
    for f in getFiles(path, 'db', filtr='MERGED_SCRAPER'):
        print f
        try:
            tweets = parseDB(f )

            c.executemany('INSERT OR IGNORE INTO tweets VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tweets )
            conn.commit()
        except Exception, e:
            print f, ':', str(e)
    
    conn.close()
    print 'Databases Merged!'
    print dbPath

In [18]:
path = '/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/all_scraper/'

In [19]:
conn = sqlite3.connect(dbPath)

In [18]:
merge_scraped(path)

/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/all_scraper/2016_02__6_twitter.db
/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/all_scraper/twitter copy.db
/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/all_scraper/twitter.db
/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/all_scraper/twitter1.db
/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/all_scraper/twitter2.db
/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/all_scraper/twitter_1.db
/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/all_scraper/twitter_1.db : database disk image is malformed
/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/all_scraper/twitter_My1.db
/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/all_scraper/twitter_My1.db : no such table: tweets
/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/all_scraper/twitter_old.db
Databases Merged!
/Users/ca

In [11]:
tweets = parseDB('/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/all_scraper/twitter2.db')

In [12]:
dbPath = '/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/CUSP_QC_twitter_scripts/data/twitter2_parsed.db'

conn = sqlite3.connect(dbPath)
c = conn.cursor()
c.execute(tweetE)

In [13]:
c.executemany('INSERT OR IGNORE INTO tweets VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tweets )
conn.commit()

In [14]:
conn.close()